In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get("HF_CREDENTIALS")

In [ ]:
!pip install datasets --quiet
!pip install emoji --quiet
!pip install evaluate --quiet
!pip install peft --quiet
!pip install accelerate --quiet
!pip install bitsandbytes --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

from datasets import load_dataset, load_from_disk, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, TrainingArguments, Trainer, DataCollatorWithPadding, RobertaTokenizer, RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import os
import torch
import torch.nn.functional as F
import random
import numpy as np
import pandas as pd
import unicodedata
import re
import emoji
import evaluate
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Settings
model_name = "jordanfan/modernBERT_suicide_base"
model_upload_name = "modernBERT_depression"
#lora_layers = ["query", "key", "value"]
# ["query", "key", "value"] # BERT
# ["query_proj", "key_proj", "value_proj"] #Deberta
# ['Wi', 'Wqkv', 'Wo'] # ModernBERT
batch_size = 24
epochs = 3
weight_decay=0.10
learning_rate = 2e-5

In [ ]:
# !pip install datasets --quiet
# from datasets import load_dataset, load_from_disk, Dataset, DatasetDict
# import pandas as pd
# from sklearn.model_selection import train_test_split

## Depression data pulled from https://github.com/KUAS-ubicomp-lab/Depression_Severity_Levels_Dataset

# df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/W210/Depression_Severity_Levels_Dataset.csv", encoding = "utf-8")
# df = df.rename(columns = {"label": "labels"})
# df = df.drop_duplicates(subset = "text")
# train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
# train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)


# dataset_dict = DatasetDict({
#     "train": Dataset.from_pandas(train_df),
#     "validation": Dataset.from_pandas(val_df),
#     "test": Dataset.from_pandas(test_df)
# })
# dataset_dict.push_to_hub("jordanfan/depression_severity_all", token= HF_TOKEN)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token = HF_TOKEN
                                          )
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels = 3,
                                                           ignore_mismatched_sizes=True,
                                                           token = HF_TOKEN
                                                           )

# Lora stopped working for modernBERT

# lora_config = LoraConfig(
#     task_type = TaskType.SEQ_CLS,
#     r=8,  # LoRA rank (lower values = less memory, faster training)
#     lora_alpha=32,  # Scaling factor
#     lora_dropout=0.1,  # Dropout for stability
#     target_modules= lora_layers
# )
# model = get_peft_model(model, lora_config)

# # Reinitialize the classification layer weights
torch.nn.init.xavier_uniform_(model.classifier.weight)
#torch.nn.init.xavier_uniform_(model.classifier.out_proj.weight) # Reinitialization for roBERTa

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/598M [00:00<?, ?B/s]

Parameter containing:
tensor([[ 0.0224,  0.0111, -0.0002,  ...,  0.0508, -0.0048, -0.0830],
        [-0.0834, -0.0729, -0.0392,  ..., -0.0520, -0.0558,  0.0523],
        [ 0.0768,  0.0521, -0.0258,  ..., -0.0314,  0.0378, -0.0155]],
       requires_grad=True)

In [ ]:
def preprocess_text(text):
    # Convert emojis to text
    text = emoji.demojize(text, delimiters=(" ", " "))
    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # Remove user mentions (@username)
    text = re.sub(r"@\w+", "", text)
    # Remove excessive special characters
    text = re.sub(r"[^\w\s,.!?;:()]", "", text)
    # Normalize spacing
    text = text.strip()
    return text

def tokenize_batch(examples):
  return tokenizer(examples['text'], max_length = 512, return_tensors = "pt", padding = True, truncation = True)

def encode_labels(examples):
  return {"labels": label2id[examples["labels"]]}


def evaluate_test(model, tokenized_data):
  training_args = TrainingArguments(
    output_dir="./results",  # Directory to store logs
    per_device_eval_batch_size=32,  # Adjust based on GPU memory
    do_train=False,  # We are only evaluating
    do_eval=True,
    fp16=True,  # Enable mixed precision for faster inference
    report_to="none",  # Avoid unnecessary logging
  )

  trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_data,
    compute_metrics=compute_metrics,
  )
  results = trainer.evaluate()
  return results

def get_probas(model, tokenized_data):
  training_args = TrainingArguments(
      output_dir="./results",  # Directory to store logs
      per_device_eval_batch_size=32,  # Adjust based on GPU memory
      do_train=False,  # We are only evaluating
      do_eval=True,
      fp16=True,  # Enable mixed precision for faster inference
      report_to="none",  # Avoid unnecessary logging
  )

  # Initialize Trainer
  trainer_mental_bert = Trainer(
      model=model,
      args=training_args
  )
  preds = trainer_mental_bert.predict(tokenized_data).predictions
  probas = torch.nn.functional.softmax(torch.tensor(preds), dim = -1)
  probas = pd.DataFrame(probas)
  probas.columns = ["none_proba", "low_proba", "moderate_severe_proba"]
  probas["index"] = tokenized_data["index"]
  probas["class"] = tokenized_data["labels"]
  return probas


In [ ]:
df = load_dataset("jordanfan/depression_severity_all")

df = df.filter(lambda x: x["text"] is not None and x["labels"] is not None and len(x["text"]) > 10)
df = df.map(lambda x: {"text": preprocess_text(x["text"])})
df = df.map(tokenize_batch, batched=True)
# Map moderate and severe to same class since we would want intervention at that severity
label2id = {"minimum": 0, "mild": 1, "moderate": 2, "severe": 2}
df = df.map(encode_labels)


README.md:   0%|          | 0.00/574 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/9.54M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/2.39M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/23336 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/5809 [00:00<?, ? examples/s]

Map:   0%|          | 0/3230 [00:00<?, ? examples/s]

Map:   0%|          | 0/23253 [00:00<?, ? examples/s]

Map:   0%|          | 0/5809 [00:00<?, ? examples/s]

Map:   0%|          | 0/3230 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred):
    """Compute Accuracy, Precision, Recall, and F1-score."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    accuracy = accuracy_score(labels, predictions)
    # Compute Loss Manually
    logits_tensor = torch.tensor(logits)
    labels_tensor = torch.tensor(labels, dtype=torch.long)
    loss = F.cross_entropy(logits_tensor, labels_tensor).item()

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "loss": loss
    }

In [ ]:
args = TrainingArguments(
    output_dir=f"jordanfan/{model_upload_name}",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size= batch_size,
    per_device_eval_batch_size= batch_size,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    learning_rate = learning_rate,
    optim = "adamw_8bit",
    load_best_model_at_end=True,
    fp16=True,
    report_to=["tensorboard"],
    push_to_hub = True,
    hub_token = HF_TOKEN,
    hub_model_id = f"jordanfan/{model_upload_name}",
)

trainer = Trainer(
    model=model,
    args= args,
    train_dataset=df["train"],
    eval_dataset=df["validation"],
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.516700,0.476247,0.789292,0.794457,0.789292,0.786698
2,0.326400,0.479697,0.800138,0.802330,0.800138,0.800230
3,0.151200,0.757705,0.790670,0.790862,0.790670,0.790444


No files have been modified since last commit. Skipping to prevent empty commit.


TrainOutput(global_step=2907, training_loss=0.31663567773000784, metrics={'train_runtime': 2893.332, 'train_samples_per_second': 24.11, 'train_steps_per_second': 1.005, 'total_flos': 2.3771091376659456e+16, 'train_loss': 0.31663567773000784, 'epoch': 3.0})

In [ ]:
trainer.create_model_card(
    language='english',
    model_name=f"{model_upload_name}",
    tasks='classification',
    tags='classification',
    dataset='jordanfan/depression_severity_all',
    dataset_args=f"Baseline"
)

commit_msg = f"Depression 3 classes"
trainer.push_to_hub(commit_message=commit_msg)

CommitInfo(commit_url='https://huggingface.co/jordanfan/modernBERT_depression/commit/979cecbb467feecf43b98c897bae969a2760cd90', commit_message='Depression 3 classes', commit_description='', oid='979cecbb467feecf43b98c897bae969a2760cd90', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jordanfan/modernBERT_depression', endpoint='https://huggingface.co', repo_type='model', repo_id='jordanfan/modernBERT_depression'), pr_revision=None, pr_num=None)

# Evaluation (Different # Classes)

In [ ]:
# 4 classes (modernBERT)
preds = trainer.predict(df["test"]).predictions
probas = torch.nn.functional.softmax(torch.tensor(preds), dim = -1)
print(classification_report(np.array(df["test"]['labels']), np.argmax(probas, axis = 1), digits = 3))

              precision    recall  f1-score   support

           0      0.846     0.901     0.873       639
           1      0.804     0.700     0.748      1049
           2      0.890     0.746     0.812       477
           3      0.710     0.823     0.762      1065

    accuracy                          0.787      3230
   macro avg      0.812     0.793     0.799      3230
weighted avg      0.794     0.787     0.787      3230



In [ ]:
# 2 classes: Combine minimum and mild, combine moderate and severe (modernBERT)
preds = trainer.predict(df["test"]).predictions
probas = torch.nn.functional.softmax(torch.tensor(preds), dim = -1)
print(classification_report(np.array(df["test"]['labels']), np.argmax(probas, axis = 1), digits = 3))

              precision    recall  f1-score   support

           0      0.856     0.738     0.793      1688
           1      0.751     0.864     0.803      1542

    accuracy                          0.798      3230
   macro avg      0.803     0.801     0.798      3230
weighted avg      0.806     0.798     0.798      3230



In [ ]:
# 3 classes: Combine moderate and severe (modernBERT)
preds = trainer.predict(df["test"]).predictions
probas = torch.nn.functional.softmax(torch.tensor(preds), dim = -1)
print(classification_report(np.array(df["test"]['labels']), np.argmax(probas, axis = 1), digits = 3))

              precision    recall  f1-score   support

           0      0.831     0.894     0.861       639
           1      0.810     0.678     0.738      1049
           2      0.771     0.832     0.800      1542

    accuracy                          0.794      3230
   macro avg      0.804     0.801     0.800      3230
weighted avg      0.795     0.794     0.792      3230



3 classes have highest precision for severe depression, train rest of models using 3 classes

In [ ]:
# 3 classes: Combine moderate and severe (mental-BERT)
preds = trainer.predict(df["test"]).predictions
probas = torch.nn.functional.softmax(torch.tensor(preds), dim = -1)
print(classification_report(np.array(df["test"]['labels']), np.argmax(probas, axis = 1), digits = 3))

              precision    recall  f1-score   support

           0      0.835     0.840     0.838       639
           1      0.800     0.713     0.754      1049
           2      0.777     0.833     0.804      1542

    accuracy                          0.795      3230
   macro avg      0.804     0.795     0.799      3230
weighted avg      0.796     0.795     0.794      3230



In [ ]:
# 3 classes: Combine moderate and severe (mental-roBERTa)
preds = trainer.predict(df["test"]).predictions
probas = torch.nn.functional.softmax(torch.tensor(preds), dim = -1)
print(classification_report(np.array(df["test"]['labels']), np.argmax(probas, axis = 1), digits = 3))

              precision    recall  f1-score   support

           0      0.862     0.831     0.846       639
           1      0.721     0.729     0.725      1049
           2      0.769     0.775     0.772      1542

    accuracy                          0.771      3230
   macro avg      0.784     0.778     0.781      3230
weighted avg      0.772     0.771     0.772      3230



## Misclassifications

In [ ]:
# modernBERT
y_test = np.array(df["test"]['labels'])
preds = np.argmax(probas, axis = 1).numpy()
misclassified_labels = y_test[y_test != preds]
misclassified_preds = preds[preds != y_test]

misclassified = pd.DataFrame({"labels": misclassified_labels, "preds": misclassified_preds})
misclassified["label|prediction"] = misclassified["labels"].astype(str) + " | " + misclassified["preds"].astype(str)
misclassified["label|prediction"].value_counts()

,count
label|prediction,
1 | 2,324
2 | 1,141
2 | 0,81
0 | 2,73
1 | 0,12
0 | 1,8


In [ ]:
#mental-BERT
y_test = np.array(df["test"]['labels'])
preds = np.argmax(probas, axis = 1).numpy()
misclassified_labels = y_test[y_test != preds]
misclassified_preds = preds[preds != y_test]

misclassified = pd.DataFrame({"labels": misclassified_labels, "preds": misclassified_preds})
misclassified["label|prediction"] = misclassified["labels"].astype(str) + " | " + misclassified["preds"].astype(str)
misclassified["label|prediction"].value_counts()

,count
label|prediction,
1 | 2,279
2 | 1,174
0 | 2,89
2 | 0,84
1 | 0,22
0 | 1,13


In [ ]:
#mental-roBERTa
y_test = np.array(df["test"]['labels'])
preds = np.argmax(probas, axis = 1).numpy()
misclassified_labels = y_test[y_test != preds]
misclassified_preds = preds[preds != y_test]

misclassified = pd.DataFrame({"labels": misclassified_labels, "preds": misclassified_preds})
misclassified["label|prediction"] = misclassified["labels"].astype(str) + " | " + misclassified["preds"].astype(str)
misclassified["label|prediction"].value_counts()

,count
label|prediction,
2 | 1,278
1 | 2,268
0 | 2,90
2 | 0,69
0 | 1,18
1 | 0,16


# Ensemble Depression Models

In [ ]:
# Load in pre-trained models
tokenizer_mental_bert = AutoTokenizer.from_pretrained("jordanfan/mental-BERT_depression", token = HF_TOKEN)
model_mental_bert = AutoModelForSequenceClassification.from_pretrained("jordanfan/mental-BERT_depression", token = HF_TOKEN, num_labels = 3, ignore_mismatched_sizes=True)
tokenizer_mental_roberta = AutoTokenizer.from_pretrained("jordanfan/mental-roberta_depression", token = HF_TOKEN)
model_mental_roberta = AutoModelForSequenceClassification.from_pretrained("jordanfan/mental-roberta_depression", token = HF_TOKEN, num_labels = 3, ignore_mismatched_sizes = True)
tokenizer_modernbert = AutoTokenizer.from_pretrained("jordanfan/modernBERT_depression")
model_modernbert = AutoModelForSequenceClassification.from_pretrained("jordanfan/modernBERT_depression", token = HF_TOKEN, num_labels = 3, ignore_mismatched_sizes = True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at djangodevloper/bert-base-sa-mental-uncased and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df = load_dataset("jordanfan/depression_severity_all")

df = df.filter(lambda x: x["text"] is not None and x["labels"] is not None and len(x["text"]) > 10)
df = df.map(lambda x: {"text": preprocess_text(x["text"])})
label2id = {"minimum": 0, "mild": 1, "moderate": 2, "severe": 2}
id2label = {0: "minimum", 1: "mild", 2: "moderate/severe"}
df = df.map(encode_labels)

val_df = df["validation"]
val_df = val_df.add_column("index", list(range(len(val_df))))
test_df = df["test"]
test_df = test_df.add_column("index", list(range(len(test_df))))

Map:   0%|          | 0/23253 [00:00<?, ? examples/s]

Map:   0%|          | 0/5809 [00:00<?, ? examples/s]

Map:   0%|          | 0/3230 [00:00<?, ? examples/s]

In [ ]:
# Tokenize test set for each model
tokenizer = tokenizer_mental_bert
#tokenized_mental_bert_val = val_df.map(tokenize_batch, batched = True)
tokenized_mental_bert_test = test_df.map(tokenize_batch, batched = True)
tokenizer = tokenizer_mental_roberta
#tokenized_mental_roberta_val = val_df.map(tokenize_batch, batched = True)
tokenized_mental_roberta_test = test_df.map(tokenize_batch, batched = True)
tokenizer = tokenizer_modernbert
#tokenized_modernbert_val = val_df.map(tokenize_batch, batched = True)
tokenized_modernbert_test = test_df.map(tokenize_batch, batched = True)

Map:   0%|          | 0/3230 [00:00<?, ? examples/s]

Map:   0%|          | 0/3230 [00:00<?, ? examples/s]

Map:   0%|          | 0/3230 [00:00<?, ? examples/s]

In [ ]:
# Get predicted probabilities for test set

probas_mental_bert = get_probas(model_mental_bert, tokenized_mental_bert_test)
probas_mental_roberta = get_probas(model_mental_roberta, tokenized_mental_roberta_test)
probas_modernbert = get_probas(model_modernbert, tokenized_modernbert_test)

probas_test = np.mean([np.array(probas_mental_bert[["none_proba", "low_proba", "moderate_severe_proba"]]),
                      np.array(probas_mental_roberta[["none_proba", "low_proba", "moderate_severe_proba"]]),
                      np.array(probas_modernbert[["none_proba", "low_proba", "moderate_severe_proba"]])],
                      axis = 0)

# Take average because metrics are all pretty similar across models
preds_test_mean = np.argmax(probas_test, axis = 1)

# Look into majority voting
def most_common(lst):
    return max(set(lst), key=lst.count)

mental_bert_preds = np.argmax(np.array(probas_mental_bert[["none_proba", "low_proba", "moderate_severe_proba"]]), axis = 1)
mental_roberta_preds = np.argmax(np.array(probas_mental_roberta[["none_proba", "low_proba", "moderate_severe_proba"]]), axis = 1)
modernbert_preds = np.argmax(np.array(probas_modernbert[["none_proba", "low_proba", "moderate_severe_proba"]]), axis = 1)

preds_test_voting = np.array(pd.DataFrame({"mental_bert_preds": mental_bert_preds,
                                      "mental_roberta_preds": mental_roberta_preds,
                                      "modernbert_preds": modernbert_preds})\
                        .apply(lambda x: most_common([x["mental_bert_preds"],
                                            x["mental_roberta_preds"],
                                            x["modernbert_preds"]]), axis = 1))

In [ ]:
# Ensembled Results - Average
print(classification_report(np.array(df["test"]['labels']), preds_test_mean, digits = 3))

              precision    recall  f1-score   support

           0      0.864     0.869     0.867       639
           1      0.815     0.722     0.765      1049
           2      0.788     0.848     0.817      1542

    accuracy                          0.811      3230
   macro avg      0.823     0.813     0.816      3230
weighted avg      0.812     0.811     0.810      3230



In [ ]:
# Ensembled Results - Voting
print(classification_report(np.array(df["test"]['labels']), preds_test_voting, digits = 3))

              precision    recall  f1-score   support

           0      0.848     0.875     0.861       639
           1      0.804     0.719     0.759      1049
           2      0.789     0.836     0.812      1542

    accuracy                          0.806      3230
   macro avg      0.814     0.810     0.811      3230
weighted avg      0.806     0.806     0.805      3230



Ensembling through averaging performs better, prioritize average model